# 1. Setting-up

1.1 Import required libraries

In [ ]:
import heapq
import string
from nltk.corpus import stopwords
import re
import itertools
from wordcloud import WordCloud
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

1.2 Loading the datasets

In [ ]:
all_series_df= pd.read_csv('all_series.csv')
people_df = pd.read_csv('people.csv')
production_companies_df = pd.read_csv('production_companies.csv')
networks_df = pd.read_csv('networks.csv')
genres_df = pd.read_csv('genres.csv')


# 2. Exploratory Data Analysis

2.1 Overview of Dataset

In [ ]:
all_series_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tmdb_id                   1515 non-null   int64  
 1   name                      1515 non-null   object 
 2   original_name             1515 non-null   object 
 3   keywords                  966 non-null    object 
 4   airing_date               1451 non-null   object 
 5   poster_img_url            1414 non-null   object 
 6   background_img_url        1179 non-null   object 
 7   directors_ids             165 non-null    object 
 8   cast_ids                  1485 non-null   object 
 9   genres_ids                1515 non-null   object 
 10  number_of_seasons         1511 non-null   float64
 11  number_of_episodes        1502 non-null   float64
 12  episode_run_time          1289 non-null   float64
 13  synopsis                  1432 non-null   object 
 14  populari

2.2 Data cleaning

*  Create a new DataFrame
*   Converts IDs to actual words



In [ ]:
##creates a new DataFrame named series by selecting specific columns
series = all_series_df[["tmdb_id", "name", "directors_ids", "cast_ids", "genres_ids", "synopsis", "networks_ids", "production_companies_ids", "airing_date", "number_of_episodes"]]

In [ ]:
#Converting IDs from genres, networks, production_companies, directors, and cast into words
def translate_ids_to_names(main_df, target_column, target_df):

    # going through each list of ids (column cell)
    for index, value in main_df[target_column].iteritems():

        new_values = []

        if not pd.isna(value):
            values = value.split(", ")

            for element in values:

                id = int(element)
                name = target_df.loc[target_df["tmdb_id"] == id, "name"].item()

                new_values.append(name)

            if not new_values:
                print("ERROR")
            else:
                new_values = ", ".join(new_values)
                main_df.at[index, target_column] = new_values

In [ ]:
#tranalste IDs to names for necessary  columns
translate_ids_to_names(series, "genres_ids", genres_df)
translate_ids_to_names(series, "networks_ids", networks_df)
translate_ids_to_names(series, "production_companies_ids", production_companies_df)
translate_ids_to_names(series, "directors_ids", people_df)
translate_ids_to_names(series, "cast_ids", people_df)

<ipython-input-95-a443a2389825>:5: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, value in main_df[target_column].iteritems():


In [ ]:
#Rename IDs Counlmns
new_columns = {"genres_ids": "genres", "networks_ids": "networks", "production_companies_ids": "production_companies", "directors_ids": "directors","cast_ids": "cast"}
series = series.rename(columns = new_columns)
series

,tmdb_id,name,directors,cast,genres,synopsis,networks,production_companies,airing_date,number_of_episodes
0,136699,Glitch,Roh Deok,"Jeon Yeo-been, Nana, Lee Dong-hwi, Ryu Kyung-s...","Drama, Comedy, Mystery, Sci-Fi & Fantasy",A young woman joins forces with a UFO enthusia...,Netflix,Studio 329,2022-10-07,10.0
1,99966,All of Us Are Dead,JQ Lee,"Park Ji-hu, Yoon Chan-young, Cho Yi-hyun, Lomo...","Action & Adventure, Drama, Sci-Fi & Fantasy",A high school becomes ground zero for a zombie...,Netflix,"Kim Jong-hak Production, SLL, Film Monster",2022-01-28,12.0
2,93405,Squid Game,Hwang Dong-hyuk,"Lee Jung-jae, Park Hae-soo, Jung Ho-yeon, Wi H...","Action & Adventure, Mystery, Drama",Hundreds of cash-strapped players accept a str...,Netflix,Siren Pictures,2021-09-17,9.0
3,112836,Money Heist: Korea - Joint Economic Area,NaN,"Yoo Ji-tae, Yunjin Kim, Park Hae-soo, Jeon Jon...","Action & Adventure, Crime, Drama, Mystery","Disguised under the shadows of a mask, a crew ...",Netflix,"Content Zium, BH Entertainment",2022-06-24,6.0
4,112888,True Beauty,NaN,"Moon Ga-young, Cha Eun-woo, Hwang In-yeop, Par...","Comedy, Drama",Lim Ju-kyung is a high school student. Since s...,tvN,"Bon Factory, Studio Dragon, Studio N",2020-12-09,16.0
...,...,...,...,...,...,...,...,...,...,...
1510,14144,Happy Woman,NaN,"Yoon Jung-hee, Jung Gyu-woon, Kim Suk-hoon, Pa...",Drama,NaN,KBS2,NaN,2007-01-06,1.0
1511,14070,Mina,NaN,Chae Jung-an,Drama,Mina was a 20-episode South Korean drama broad...,KBS2,NaN,2001-11-05,16.0
1512,12062,Salut D'Amour,NaN,Sung Hyun-ah,"Drama, Comedy","Salut D'Amour, despite its French name, is a S...",KBS2,NaN,1994-11-01,1.0
1513,212594,She Makes My Heart Flutter,NaN,NaN,Drama,"Two women, one outgoing and one shy, work toge...",NaN,SOO NOT SUE studio,2022-10-20,1.0


2.3 Handling Missing/Duplicate Values

In [ ]:
#filling missing values

for column in series.columns:
    if series[column].dtype == 'object':
        mode_value = series[column].mode()[0]
        series[column].fillna(mode_value, inplace=True)
    elif series[column].dtype == 'float64':
        series[column].fillna(series[column].mean(), inplace=True)
    elif series[column].dtype == 'int64':
        series[column].fillna(series[column].median(), inplace=True)
    else:
      pass


In [ ]:
#checks for any missing values/
series.isna().sum()

tmdb_id                 0
name                    0
directors               0
cast                    0
genres                  0
synopsis                0
networks                0
production_companies    0
airing_date             0
number_of_episodes      0
dtype: int64

In [ ]:
new_series = series[["tmdb_id", "name", "cast", "genres", "synopsis","airing_date","networks","number_of_episodes"]]


In [ ]:
#Check for Duplicate
new_series.duplicated().sum()

0

3.3 Checking Data type for a new DataFrame

In [ ]:
print(f"Shape: {new_series.shape}")
print(f"Time Period: {new_series['airing_date'].min()}-{new_series['airing_date'].max()}")
print("Data Dictionary: ")
new_series.info()

Shape: (1515, 8)
Time Period: 1983-03-31-2023-06-30
Data Dictionary: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tmdb_id             1515 non-null   int64  
 1   name                1515 non-null   object 
 2   cast                1515 non-null   object 
 3   genres              1515 non-null   object 
 4   synopsis            1515 non-null   object 
 5   airing_date         1515 non-null   object 
 6   networks            1515 non-null   object 
 7   number_of_episodes  1515 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 94.8+ KB


3.1 Exploring Genres of K-Drama

In [ ]:
# Categorical Data Analysis
# Genre distribution
genre_counts = new_series['genres'].str.split(',').explode().str.strip().value_counts()


In [ ]:
# Network distribution
network_counts = new_series['networks'].str.split(',').explode().str.strip().value_counts()

3.3 analysis for how many K-Drama has been released in each year

3.4 Filtering Recommendation
  * For first-time users, they can filter K-Drama by desired number of episodes to try out K-Drama.

In [ ]:
#Filtering Drama by Number of Episode
epi = int(input("Enter your atmost number of episodes:"))
filtered_dramas = new_series[new_series['number_of_episodes'] <= epi]
top_dramas = filtered_dramas.sort_values(by=['name','number_of_episodes'], ascending=[False, False]).head(5)

# Selecting specific columns
selected_columns = ['year',
       'number_of_episodes', 'networks',
       'synopsis', 'cast', 'genres', ]

# Displaying the top 5 rows with selected columns
result = top_dramas[selected_columns]
result.head()

Enter your atmost number of episodes:12


,year,number_of_episodes,networks,synopsis,cast,genres
1416,2019,5.0,MBC,Tea master Hong won the Winter Course Competit...,"Park Ji An, Shin Hyun Jin, Lee Yoo Ri",Drama
226,2020,12.0,"KBS2, wavve",Kim Moo-Young has been a zombie for the past 2...,"Choi Jin-hyuk, Park Ju-hyun, Hwang Bo-ra, Kwon...","Sci-Fi & Fantasy, Mystery, Comedy, Drama"
1299,2021,1.0,tvN,About a prince suffering from a mysterious cur...,"Park Hyung-sik, Jeon So-nee, Pyo Ye-jin, Yun J...","Mystery, Drama"
305,2021,12.0,"KBS2, wavve","Set in 1980s during the Gwangju Uprising, Yout...","Lee Do-hyun, Go Min-si, Lee Sang Yi, Geum Sae-...",Drama
1313,2021,10.0,MBC,"Centered around a secret composer, fans, and f...","Yoon San-ha, Park Su-min, Han Gi-chan",Drama


3.5 exploring top 50 common words in Synopsis
